In [1]:
from sklearn.linear_model import LogisticRegression
import tpot2
import numpy as np
import copy
from tqdm import tqdm, tnrange, tqdm_notebook
import sklearn
import sklearn.metrics
import sklearn.datasets
import tpot2 

def params_LogisticRegression(trial, name=None):
    params = {}
    params['solver'] = trial.suggest_categorical(name=f'solver_{name}',
                                                 choices=[f'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
    params['dual'] = False
    params['penalty'] = 'l2'
    params['C'] = trial.suggest_float(f'C_{name}', 1e-4, 1e4, log=True)
    params['l1_ratio'] = None
    if params['solver'] == 'liblinear':
        params['penalty'] = trial.suggest_categorical(name=f'penalty_{name}', choices=['l1', 'l2'])
        if params['penalty'] == 'l2':
            params['dual'] = trial.suggest_categorical(name=f'dual_{name}', choices=[True, False])
        else:
            params['penalty'] = 'l1'

    params['class_weight'] = trial.suggest_categorical(name=f'class_weight_{name}', choices=['balanced'])
    param_grid = {'solver': params['solver'],
                  'penalty': params['penalty'],
                  'dual': params['dual'],
                  'multi_class': 'auto',
                  'l1_ratio': params['l1_ratio'],
                  'C': params['C'],
                  'n_jobs': 1,
                  }
    return param_grid



import tpot2
import sklearn
import sklearn.datasets

est = tpot2.TPOTEstimatorSteadyState(  population_size=10,
                                        initial_population_size = 10,
                                        max_queue_size = 10,
                                        min_individuals_finished = 1,
                                        max_evaluated_individuals = 50,
                            scorers=['roc_auc_ovr',tpot2.model_complexity_scorer], #scorers can be a list of strings or a list of scorers. These get evaluated during cross validation. 
                            scorers_weights=[1,-1],
                            classification=True,
                            root_config_dict =  tpot2.make_classifier_config_dictionary(),
                            #inner_config_dict= "arithmetic_transformer",
                            #leaf_config_dict="feature_set_selector",
                            n_jobs=10,

                            early_stop=None, #how many generations with no improvement to stop after
                            
                            #List of other objective functions. All objective functions take in an untrained GraphPipeline and return a score or a list of scores
                            other_objective_functions= [],
                            
                            #List of weights for the other objective functions. Must be the same length as other_objective_functions. By default, bigger is better is set to True. 
                            other_objective_functions_weights=[],
                            processes=True,
                            verbose=5,
                            max_eval_time_seconds=30,
                            max_time_seconds=600,
                            memory_limit = "1GB",
                            )

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_digits(return_X_y=True)
X, y = sklearn.datasets.make_classification(100000,10)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)
est.fit(X_train, y_train)
print(scorer(est, X_test, y_test))

2023-05-23 07:18:34,312 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-y650rouh', purging
2023-05-23 07:18:34,312 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-o925tg1r', purging
2023-05-23 07:18:34,312 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vccn3one', purging
2023-05-23 07:18:34,313 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-co6bdx7l', purging
2023-05-23 07:18:34,313 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yx45p3er', purging
2023-05-23 07:18:34,313 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-t_hg9tsc', purging
2023-05-23 07:18:34,313 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cch7fzs8', purging

WARNING THIS INDIVIDUAL CAUSED AND EXCEPTION 
 ['MultinomialNB_1'] 
 Negative values in data passed to MultinomialNB (input X) 
 Traceback (most recent call last):
  File "/home/ribeirop/Projects/TPOT_Dev/tpot2/tpot2/objectives.py", line 53, in objective_nan_wrapper
    value = func_timeout.func_timeout(timeout, objective_function, args=[individual], kwargs=objective_kwargs)
  File "/home/ribeirop/miniconda3/envs/tpot2env/lib/python3.10/site-packages/func_timeout/dafunc.py", line 108, in func_timeout
    raise_exception(exception)
  File "/home/ribeirop/miniconda3/envs/tpot2env/lib/python3.10/site-packages/func_timeout/py3_raise.py", line 7, in raise_exception
    raise exception[0] from None
  File "/home/ribeirop/Projects/TPOT_Dev/tpot2/tpot2/experimental/steady_state_estimator.py", line 662, in objective_function
    return objective_function_generator(
  File "/home/ribeirop/Projects/TPOT_Dev/tpot2/tpot2/experimental/steady_state_estimator.py", line 1064, in objective_function_gene

Evaluations:   6%|▌         | 3/50 [00:31<09:04, 11.57s/it]

WARNING AN INDIVIDUAL TIMED OUT: 
 ['GradientBoostingClassifier_1'] 

WARNING AN INDIVIDUAL TIMED OUT: 
 ['RandomForestClassifier_1'] 

WARNING AN INDIVIDUAL TIMED OUT: 
 [('GaussianNB_1', 'FastICA_1')] 

Best roc_auc_score score: 0.963505707475918
Best model_complexity_scorer score: 4.0


Evaluations:  10%|█         | 5/50 [00:32<04:21,  5.81s/it]

...

Evaluations:  50%|█████     | 25/50 [01:24<01:22,  3.28s/it]

Best roc_auc_score score: 0.9758387331401523
Best model_complexity_scorer score: 2.0


Evaluations:  54%|█████▍    | 27/50 [01:40<01:47,  4.66s/it]

Best roc_auc_score score: 0.9758387331401523
Best model_complexity_scorer score: 2.0
WARNING THIS INDIVIDUAL CAUSED AND EXCEPTION 
 [('MultinomialNB_1', 'FeatureAgglomeration_1')] 
 Negative values in data passed to MultinomialNB (input X) 
 Traceback (most recent call last):
  File "/home/ribeirop/Projects/TPOT_Dev/tpot2/tpot2/objectives.py", line 53, in objective_nan_wrapper
    value = func_timeout.func_timeout(timeout, objective_function, args=[individual], kwargs=objective_kwargs)
  File "/home/ribeirop/miniconda3/envs/tpot2env/lib/python3.10/site-packages/func_timeout/dafunc.py", line 108, in func_timeout
    raise_exception(exception)
  File "/home/ribeirop/miniconda3/envs/tpot2env/lib/python3.10/site-packages/func_timeout/py3_raise.py", line 7, in raise_exception
    raise exception[0] from None
  File "/home/ribeirop/Projects/TPOT_Dev/tpot2/tpot2/experimental/steady_state_estimator.py", line 662, in objective_function
    return objective_function_generator(
  File "/home/ribe

: 